Summer school on Finite Elements
===
## Universidad Nacional Agraria La Molina
### December, 2018

```m
Carlos
```

u es la solución explícitva

In [1]:
%%file createmesh.m
function [mesh] = createmesh(DIM,M)
%CREATEMESH creates a regular finite element mesh in DIM dimensions
%   M is the number of elements in every mesh direction
%   therefore, h=1/M is the mesh size
%   
%   The mesh has (M+1) points in every line
%     0 on the boundary, 1,2,...,M-1 innerpoint and M on the boundary
%   The total number of points is N = (M+1)^DIM
%
%   The mesh is a N x DIM array.
%   The coordinate of point i is: mesh(i,0), mesh(i,1)            (in 2d)
%                            and: mesh(i,0), mesh(i,1), mesh(i,2) (in 3d)

h=1/M;                % compute mesh size
N=(M+1)^DIM;          % number of points
mesh=zeros(N,DIM);
if DIM==1
    mesh = h*[0:1:M];
elseif DIM==2
    mesh(:,1) = repmat(h*[0:1:M]',M+1,1);
    mesh(:,2) = repelem(h*[0:1:M]',M+1,1); % h*kron([0:1:M]', ones(M+1,1));
elseif DIM==3
    mesh(:,1) = repmat(h*[0:1:M]',(M+1)*(M+1),1);
    mesh(:,2) = repmat(repelem(h*[0:1:M]',(M+1),1),(M+1),1);
    mesh(:,3) = repelem(h*[0:1:M]',(M+1)*(M+1),1); % h*kron([0:1:M]', ones(M+1,1));
end

Created file '/home/carlosal1015/GitHub/Finite-element-method-FEM/2018/Notebooks/createmesh.m'.


In [2]:
%%file assemblematrix2d.m
function [A] = assemblematrix2d(mesh)
%Assemble the Finite Element Matrix in 2D
%The matris given by its stencil, a 3x3 matrix
%
[N,DIM] = size(mesh);
assert(DIM==2);           % check that we are in 2d

M = N^(1/DIM);            % number of points in every direction
h = 1/(M-1);              % element size

S = [ -1/3, -1/3, -1/3 ; -1/3,  8/3, -1/3 ;  -1/3, -1/3, -1/3];


% First, we create an empty sparse matrix
A=sparse(N,N);

% Write matrix stencil into Matrix A, row by row

for my=1:M                  % row of the mesh
    for mx=1:M              % column of the mesh 
        
        % index of the row in the matrix belonging to meshpoint my,mx
        rowA = (my-1)*M + mx;
        
        % enter the diagonal
        A(rowA,rowA) = S(2,2);
        % enter left and right - if not mx=1 or mx=M
        if (mx>1) A(rowA,rowA-1) = S(2,1); end
        if (mx<M) A(rowA,rowA+1) = S(2,3); end
        % enter up and down - if not my=1 or my=M
        if (my>1) A(rowA,rowA-M) = S(1,2); end
        if (my<M) A(rowA,rowA+M) = S(3,2); end
        % enter diagonals - if not ...
        if (mx>1) && (my>1) A(rowA,rowA-1-M) = S(1,1); end
        if (mx<M) && (my>1) A(rowA,rowA+1-M) = S(1,3); end
        if (mx>1) && (my<M) A(rowA,rowA-1+M) = S(3,1); end
        if (mx<M) && (my<M) A(rowA,rowA+1+M) = S(3,3); end
    end
end

%%% set dirichlet values
for m=1:M        %
    % bottom boundary
    A(m,:) = 0;
    A(m,m) = 1;
    % left boundary
    A((m-1)*M+1,:) = 0;
    A((m-1)*M+1,(m-1)*M+1) = 1;
    % top boundary
    A((M-1)*M+m,:) = 0;
    A((M-1)*M+m,(M-1)*M+m) = 1;
    % right boundary
    A(M*m,:) = 0;
    A(M*m,M*m) = 1;
end
    
end

Created file '/home/carlosal1015/GitHub/Finite-element-method-FEM/2018/Notebooks/assemblematrix2d.m'.


In [3]:
%%file assemblerhs.m
function [rhs] = assemblerhs(mesh)

[N,DIM] = size(mesh);     % nr. of nodes and dimension

assert(DIM==2);           % make sure that we are in 2d

N1d=N^(1/DIM);            % points in every direction
M = N1d-1;                % elements in every direction
h=1/M;                    % grid size

rhs=zeros(N,1);           % vector of zero's to carry right hand side

% Gauss Points & Guass Weights
gp = h*[0.5-0.5/sqrt(3.0),0.5-0.5/sqrt(3.0);
        0.5+0.5/sqrt(3.0),0.5-0.5/sqrt(3.0);
        0.5-0.5/sqrt(3.0),0.5+0.5/sqrt(3.0);
        0.5+0.5/sqrt(3.0),0.5+0.5/sqrt(3.0)];
gw = h*h*[0.25,0.25,0.25,0.25];

% The for test functions
PHI = [(1-gp(:,1)/h).*(1-gp(:,2)/h) ,(gp(:,1)/h).*(1-gp(:,2)/h),(1-gp(:,1)/h).*(  gp(:,2)/h) ,(  gp(:,1)/h).*(  gp(:,2)/h)];


% loop over all mesh elements in y- and x-direction
% numerical quadrature with a 2x2-point Gauss rule
for my=1:M                  % row of the mesh element
    for mx=1:M              % column of the mesh element
        
        row = (my-1)*N1d + mx;  % index in lower/left corner
        x=mesh(row,1);     % x-coordinate of this point
        y=mesh(row,2);     % y-coordinate of this point
        
        % Assemble local right hand side vector
        %   for the 4 test functions in the element
        bloc=zeros(4,1);  
        for j=1:4         % loop over the four test functions of the element
            for k=1:4         % loop over the gauss points
                X=x+gp(k,1);
                Y=y+gp(k,2);
                bloc(j) = bloc(j)+gw(k)*righthandside(X,Y)*PHI(j,k); % <<<--- here's the rhs
            end
        end
        
        % enter local vector to global vector
        rhs(row)       = rhs(row)       + bloc(1);
        rhs(row+1)     = rhs(row+1)     + bloc(2);
        rhs(row+N1d)   = rhs(row+N1d)   + bloc(3);
        rhs(row+N1d+1) = rhs(row+N1d+1) + bloc(4);
    end
end
    
% set Dirichlet values
for m=1:N1d
    % 
    rhs(m) = 0;     % <<<--- here's the Dirichlet values
    rhs((m-1)*N1d+1) = 0;
    rhs(m*N1d) = 0;
    rhs((N1d-1)*N1d+m) = 0;
end

end


Created file '/home/carlosal1015/GitHub/Finite-element-method-FEM/2018/Notebooks/assemblerhs.m'.


In [ ]:
%%file plotsolution.m
function plotsolution(name,mesh,x)
%PLOTSOLUTION Summary of this function goes here
%   Detailed explanation goes here
figure('Name',name);       % create a matlab figure

[N,DIM] = size(mesh);      % mesh nodes and dimension
M=N^(1/DIM);               % nodes in every direction

h=1/(M-1);                 % grid size

if (DIM==2)
    MX = reshape(mesh(:,1),M,M);
    MY = reshape(mesh(:,2),M,M);
    Z  = reshape(x,M,M);
    surf(MX,MY,Z);
elseif (DIM==3)
    [MX,MY,MZ]=meshgrid([0:h:1]);
    Z=zeros(N+1,N+1,N+1);
    Z(2:N,2:N,2:N)=reshape(x,N-1,N-1,N-1);
    xslice=[0.3,0.7];  yslice=0.5; zslice=0.3;
    slice(MX,MY,MZ,Z,xslice,yslice,zslice);
end
end

In [ ]:
%%file start.m
function start()
% M is the number of mesh elements in every direction
% N=(M+1)*(M+1) is the total number of points 
M = 300; %M = 20

% create the mesh
mesh = createmesh(2,M);
%mesh
%pause

% assemble the matrix
A = assemblematrix2d(mesh);

% set the right hand side
b = assemblerhs2d(mesh);

% solve the linear system
x = A\b;

% show the solution
plotsolution('solution',mesh,x);
plotsolution('rhs',mesh,x); % Visualizar la función
end
% righthandside.m